<a href="https://colab.research.google.com/github/lns0801/stock/blob/main/stock.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

def get_ticker_by_company_name(company_name, download_all=False):
    """
    yfinance를 통해 S&P 500 종목 중 특정 회사명을 통해 티커를 찾는 함수
    download_all이 True로 설정되면 전체 S&P 500 종목 데이터를 다시 가져옵니다.
    """
    # 직접 S&P 500 티커 리스트를 사용하든지 yfinance 속 메서드 by 코드를 활용할 수 있습니다.
    # 여기서는 S&P 500 데이터를 한번 다운로드한다는 가정 하에 하드코딩 리스트 처리.
    sp500_tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'FB', 'TSLA', 'BRK-B', 'TSM', 'V', 'JNJ']  # 예시로 일부만 포함

    for ticker in sp500_tickers:
        company_info = yf.Ticker(ticker).info
        if 'longName' in company_info and company_name.lower() in company_info['longName'].lower():
            return ticker
    return None

def get_stock_data(tickers, start_date, end_date):
    """여러 주가 데이터를 가져오는 함수"""
    stock_data = yf.download(tickers, start=start_date, end=end_date)['Adj Close']
    return stock_data

def calculate_returns(stock_data):
    """수익률을 계산하는 함수"""
    returns = stock_data.pct_change().dropna()
    return returns

def rebalance_portfolio(weights, returns):
    """목표 비중으로 리밸런싱하는 함수 및 수익률 기여도 계산"""
    individual_contributions = (weights * returns).sum(axis=1)
    total_value = (1 + individual_contributions).cumprod()
    contribution_percentage = returns.mul(weights, axis=1).mean()

    return total_value, contribution_percentage

def run_simulation(company_names, weights, start_date, end_date, initial_investment, rebalance_period='ME'):
    tickers = [get_ticker_by_company_name(name) for name in company_names]
    tickers = [ticker for ticker in tickers if ticker]  # 유효한 티커만 포함

    if not tickers:
        print("유효한 티커를 찾을 수 없습니다.")
        return None, None

    # 주가 데이터 가져오기
    stock_data = get_stock_data(tickers, start_date, end_date)

    # 수익률 계산
    returns = calculate_returns(stock_data)

    # 초기 투자금으로 투자 시작
    portfolio_value = pd.Series(initial_investment, index=returns.index)

    # 포트폴리오 가치 기록을 위한 데이터 프레임 초기화
    portfolio_history = pd.DataFrame(index=returns.index)

    contribution_percentages = []

    for each in pd.date_range(start_date, end_date, freq=rebalance_period):
        # 타임존 정보 제거
        period_start = pd.Timestamp(each).tz_localize(None).tz_localize('UTC')
        period_end = (period_start + pd.DateOffset(months=1) - pd.DateOffset(days=1))

        period_returns = returns.loc[period_start:period_end]

        # 리밸런싱 후의 포트폴리오 가치 계산
        if not period_returns.empty:
            portfolio_value, contribution_percentage = rebalance_portfolio(weights, period_returns)
            portfolio_value *= portfolio_value.iloc[-1]
            portfolio_history = pd.concat([portfolio_history, portfolio_value], axis=0)
            contribution_percentages.append(contribution_percentage)

    # 월별 기여도를 데이터프레임으로 변환, 인덱스를 DatetimeIndex로 유지
    monthly_contributions = pd.DataFrame(contribution_percentages, index=pd.date_range(start_date, end_date, freq=rebalance_period))

    # 월별 평균 기여도 계산, .dt accessor 사용
    avg_contribution_percentage = monthly_contributions.groupby(monthly_contributions.index.month).mean()

    return portfolio_history, avg_contribution_percentage

In [ ]:

# 사용 예제
company_names = ['Apple Inc', 'Microsoft Corporation', 'Alphabet Inc']
weights = np.array([0.4, 0.3, 0.3])  # 각 종목의 목표 비중
start_date = '2000-01-01'
end_date = '2023-01-01'
initial_investment = 1000  # 초기 투자금 ($1000)

# 투자 시뮬레이션 및 리밸런싱
portfolio_history, avg_contribution_percentage = run_simulation(company_names, weights, start_date, end_date, initial_investment)

# 결과 출력 및 시각화
if portfolio_history is not None:
    print(avg_contribution_percentage)
    fig, ax1 = plt.subplots(figsize=(10, 6))

    # 포트폴리오 가치 로그 차트
    ax1.plot(portfolio_history, label='Portfolio Value', color='b')
    ax1.set_yscale('log')
    ax1.set_xlabel('Date')
    ax1.set_ylabel('Portfolio Value ($)')
    ax1.set_title('Portfolio Value Over Time (Log Scale)')
    ax1.grid(True, which='both', linestyle='--', linewidth=0.5)

    # 수익률 기여도 그래프
    ax2 = ax1.twinx()
    avg_contribution_percentage.plot(kind='bar', ax=ax2, alpha=0.3, color='gray', width=0.5)
    ax2.set_ylabel('Average Monthly Contribution (%)')
    ax2.set_ylim(0, max(avg_contribution_percentage.max()) * 1.2)
    ax2.grid(False)

    plt.show()

[*********************100%***********************]  3 of 3 completed


ValueError: Length of values (221) does not match length of index (276)